In [2]:
# import libraries
import sys
import os
import json
import requests
import random
from sqlalchemy import create_engine
from datetime import datetime, timedelta

sys.path.append("c:/Users/Owuraku/OneDrive/Documents/hybrid-trafing-pipeline/")
os.chdir("c:/Users/Owuraku/OneDrive/Documents/hybrid-trafing-pipeline/")

In [3]:
%load_ext autoreload
%load_ext sql

In [4]:
engine = create_engine("sqlite:///api/EAtrades.db")

In [5]:
%sql engine

In [7]:
%%sql 
PRAGMA journal_mode=WAL

Running query in 'sqlite:///api/EAtrades.db'

journal_mode
wal


In [6]:
%%sql
SELECT *
FROM sqlite_schema

Running query in 'sqlite:///api/EAtrades.db'

type,name,tbl_name,rootpage,sql
table,trades,trades,2,"CREATE TABLE trades ( position_id INTEGER NOT NULL, ticket INTEGER, symbol VARCHAR, order_type VARCHAR, volume FLOAT, entry_price FLOAT, timestamp DATETIME, status VARCHAR, exit_price FLOAT, profit FLOAT, exit_time DATETIME, PRIMARY KEY (position_id))"
index,ix_trades_position_id,trades,3,CREATE INDEX ix_trades_position_id ON trades (position_id)
index,ix_trades_ticket,trades,4,CREATE INDEX ix_trades_ticket ON trades (ticket)
index,ix_trades_symbol,trades,5,CREATE INDEX ix_trades_symbol ON trades (symbol)
index,ix_trades_order_type,trades,7,CREATE INDEX ix_trades_order_type ON trades (order_type)


In [8]:
%%sql
SELECT count(*) as total
FROM trades

Running query in 'sqlite:///api/EAtrades.db'

total
470


In [7]:
%%sql
SELECT *
FROM trades

Running query in 'sqlite:///api/EAtrades.db'

position_id,ticket,symbol,order_type,volume,entry_price,timestamp,status,exit_price,profit,exit_time


# Request

In [11]:
API_URL = "http://127.0.0.1:8008/trades/bulk_imports/"

In [9]:
# bult upload
symbols = ["EURUSD", "USDCAD", "AUDCHF", "EURGBP"]
num_obs = 470
data = []
counter = 0
for i in range(num_obs):
    position_id = int("".join(str(random.randint(0, 9)) for _ in range(7)))
    ticket = int("".join(str(random.randint(0, 9)) for _ in range(10)))
    symbol = random.choice(symbols)
    order_type = random.choice(random.choices(["BUY", "SELL"], weights=[0.6, 0.4], k=10))
    volume = round(random.uniform(0.1, 1.0), 2)
    entry_price = round(random.uniform(1.1000, 1.2000), 5)
    entry_time = datetime.now() - timedelta(hours=(random.randint(5, 60)))
    status = random.choice(random.choices(["OPEN", "CLOSED"], weights=[0.67, 0.33], k=20))

    profit_val = round(random.uniform(-300.0, 850.0), 2)
    if status == "CLOSED":
        exit_price = round(entry_price + (profit_val / 10000), 5)
        profit = profit_val
        exit_time = (entry_time + timedelta(minutes=(random.randint(120, 800)))).isoformat() + "Z"
    else:
        exit_price = None
        profit = None
        exit_time = None

    data.append({
        "position_id": position_id,
        "ticket": ticket,
        "symbol": symbol,
        "order_type": order_type,
        "volume": volume,
        "entry_price": entry_price,
        "timestamp": entry_time.isoformat() + "Z",
        "status": status,
        "exit_time": exit_time,
        "exit_price": exit_price,
        "profit": profit
    })

    counter += 1

print(f"Total number of records: {counter}")        
            

Total number of records: 470


In [11]:
data[0]

{'position_id': 7400024,
 'ticket': 7948374914,
 'symbol': 'USDCAD',
 'order_type': 'BUY',
 'volume': 0.93,
 'entry_price': 1.19584,
 'timestamp': '2025-12-20T19:46:55.080186Z',
 'status': 'OPEN',
 'exit_time': None,
 'exit_price': None,
 'profit': None}

In [12]:
results = requests.post(API_URL, json=data)
print(results.status_code)
print(results.json())

200
{'message': 'Successfullly inserted 470 new trades records'}
